# Modello CNN
Genereazione di mappe di perfusione cerebrale rCBV a partire da immagini di Risonanza Magnetica DSC utilizzando un modello CNN.

In [1]:
!pip install numpy torch torchvision torchio matplotlib torchinfo torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.8/172.8 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.5/237.5 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 6.9 MB/s eta 0:00:00
  Attempting uninstall: rich
    Found existing installation: rich 13.3.4
    Uninstalling rich-13.3.4:
      Successfully uninstalled rich-13.3.4


In [2]:
# imports
import torch
import torchio as tio
import numpy as np
import os
from torchinfo import summary
import skimage #serve per elaborazione immagini (nel nostro caso usiamo per le metriche)
import torch.nn as nn #ci sono moduli per alcuni modelli neurali, regressione, convoluzione ecc
import torch.nn.functional as F #qui invece ci possono essere buoni comandi per kernel, filtraggi, max pooling e cose così
import shutil
import torchvision
from torchvision import models, transforms as tt
import matplotlib.pyplot as plt
import nibabel as nib #libreria per la gestion di file .nii
from skimage.metrics import mean_squared_error as mse
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import normalized_root_mse as nrmse
from google.colab import drive
drive.mount('/content/drive/') #Ora necessario caricare su drive le cartelle, necessario però un metodo alternativo (caricare i file dal locale) COME?

Mounted at /content/drive/


In [3]:
data_dir = '/content/drive/MyDrive/Data (prova)'  # directory della cartella "data" (già importata su drive)

# Creazione di una lista di patients_ID
patients_ID = [] 
for f in os.scandir(data_dir): # restituisce un iteratore sugli elementi presenti nella directory "data_dir"
  if f.is_dir(): # se l'elemento è una directory, allora viene aggiungo alla lista
    patients_ID.append(f.name) # riempimento della lista patients_ID con tutti i nomi delle directory che sono contenuti in data e soddisfano l'if

In [4]:
#data = '/content/drive/MyDrive/Data (prova)'  # directory
#patients_ID = []
#for f in os.scandir(data):
#  if f.is_dir():
#    patients_ID.append(f.name) # riempimento della lista patients_ID con tutti i nomi delle directory che sono contenuti in data e soddisfano l'if

In [ ]:
# Creazione di una lista patients_list
patients_list = []
for current in patients_ID:
  subj_dir = data_dir +'/'+ current 
  
  #Creazione di un'istanza
  patient = tio.Subject( # creo un oggetto "Subject" per ogni paziente della lista utilizzando le immagini DSC e rCBV
      DSC = tio.ScalarImage(subj_dir + '/DSC.nii'), #.to(torch.float32), # ScalarImagine: immagine scalare 3D
      rCBV = tio.ScalarImage(subj_dir + '/DSC_ap-rCBV.nii'),  # ogni paziente contiene nome, immagine DSC e immagine rCBV
      brain_mask = tio.LabelMap(subj_dir + '/brain_mask.nii')
      )
  
  patient.DSC.set_data(patient.DSC.data.to(torch.float32))
  patient.rCBV.set_data(patient.rCBV.data.to(torch.float32))
 
  patients_list.append(patient)    # aggiunta di patient alla lista patients_list


L'oggetto "patient" rappresenta un paziente specifico, contenente il nome del paziente e le immagini DSC e rCBV del paziente.

In [ ]:
#Creazione Dataset
dataset = tio.SubjectsDataset(patients_list)
print('Dataset size:', len(dataset), 'patients')

In [ ]:
print(dataset[0]["DSC"][tio.DATA].dtype),
print(dataset[0]["rCBV"][tio.DATA].dtype)

In [ ]:
!sudo apt-get install tree
#!tree -d /content/drive/MyDrive/Data # comando per visualizzare la struttura del dataset (usa !tree per visualizzare anche le sotto-directory)

In [ ]:
# Funzione utile per visualizzare una rappresentazione testuale del dataset (utile per verifca)
# print(dataset.dry_iter())

In [ ]:
#for i in range(len(dataset)):
#  print(f"paziente {i}:")
#  subject = dataset[i]
#  for key, image in subject.items():
#    tensor = image.data
#    print(f"  {key}: {tensor.shape}")

In [ ]:
#creazione mask
#  rcbv_img=nib.load(subj_dir + "/DSC_ap-rCBV.nii")
  # converte l'immagine rCBV in un tensore PyTorch
#  rcbv_tensor = torch.from_numpy(rcbv_img.get_fdata())
#  # crea la brain_Mask selezionando i valori maggiori di 0
#  brain_mask = torch.where(rcbv_tensor > 0, torch.tensor([1]), torch.tensor([0]))
#  # crea un oggetto Nifti1Image a partire dal tensore PyTorch della brain_mask
#  brain_mask_nifti = nib.Nifti1Image(brain_mask.numpy(), rcbv_img.affine)
#  # salva l'immagine della brain_mask come file NIfTI-1 nella cartella
#  nib.save(brain_mask_nifti, subj_dir + '/brain_mask.nii')


#brain_mask = tio.LabelMap(subj_dir + '/brain_mask.nii')
#patient_x.brain_mask.plot()
#tio.ZNormalization(masking_method='brain_mask'),

### Visualizzare tipo e immagine input output

In [ ]:
#dataset[1]["DSC"].plot()
#dataset[1]["rCBV"].plot()
# dataset[1].plot() #mostra direttamente entrambe le immagini

In [ ]:
# verifica su un paziente delle proprietà delle immagini
#patient_x = dataset[1] # numero casuale 
#print('Shape of DSC images', patient_x.DSC.shape)
#print('Size of DSC images', patient_x.DSC.spacing)
#print('Shape of rCBV images', patient_x.rCBV.shape)
#print('Size of rCBV images', patient_x.rCBV.spacing)
#print('Size of brain mask', patient_x.brain_mask)
#patient_x.brain_mask.plot()

In [ ]:
#Verifica lo shape delle immagini
#print('Shape of DSC images', training_set[0]["DSC"][tio.DATA].shape)
#print('Shape of rCBV images', training_set[0]["DSC"][tio.DATA].shape)


In [ ]:
#visualizzare un dataset
#paziente_1 = dataset[0]
#paziente_1.plot() #vedo sia DSC che rCBV
#patient.DSC.plot() #vedo solo DSC plot
#patient.rCBV.plot() #vedo solo rCBV plot

## Training e Validation set

### Preparazione dei dati (trasformazioni)

In [ ]:
training_transform = tio.Compose([
    tio.ToCanonical(),
    tio.CropOrPad((120,120,80)), #Crop the images w,h,d and centre around the mask_name
    tio.Resample((2,2,2)),
    #tio.ZNormalization(masking_method='brain_mask'),
    tio.OneHot(),
])

validation_transform = tio.Compose([
    tio.ToCanonical(),
    tio.CropOrPad((120,120,80)), #Crop the images w,h,d and centre around the mask_nam
    tio.Resample((2,2,2)),
    #tio.ZNormalization(masking_method='brain_mask'),
    tio.OneHot(),
])


### Divisione del dataset

In [ ]:
# suddivisione del dataset, in modo casuale, in dati di training (70% dei dati) e di validation (30% dei dati). #MANCA 10% PER IL TESTING!!!
training_division = 0.7   # definizione della % di dati di training
validation_division = 0.2 # definizione della % di dati di validation
#test_division = 0.1

num_patients = len(dataset) # pazienti totali
train_patients = int(training_division*num_patients)
val_patients = int(validation_division*num_patients)
test_patients = int(num_patients-(train_patients+val_patients))

# creazione dei tre sottoinsiemi 
patients_division = train_patients, val_patients, test_patients
train, val, test = torch.utils.data.random_split(patients_list, patients_division) # suddivido casualmente la lista di pazienti in tre sottoinsiemi

training_set = tio.SubjectsDataset(train, transform=training_transform)
print('train_set:', len(training_set)) # per verificare il numero di dati di training
validation_set = tio.SubjectsDataset(val, transform=validation_transform)
print('val_set:', len(validation_set)) # per verificare il numero di dati di validation
test_set = tio.SubjectsDataset(test)
print('test_set:', len(test_set))

In [ ]:
training_set[0]["DSC"][tio.DATA].shape, training_set[0]["rCBV"][tio.DATA].shape

###Mask

In [ ]:
#def masking(dataset):
#  for i in range(len(dataset)):
#    mask = dataset[i]["DSC"][tio.DATA] > 0
#   print(mask.shape)
#    dataset[i]["mask"] = mask 

In [ ]:
#masking(dataset)

#dataset[0]["mask"].plot()
#dataset[1]["mask"].plot()

In [ ]:
 #visualizzare info
#print(paziente_1.DSC.shape)
#mask = dataset[0]["DSC"][tio.DATA]>0
#print(mask.shape)

#print(paziente_1.DSC)
#print(paziente_1.rCBV)

##Creazione del DataLoader

In [ ]:
#check num_cores disponibili
import multiprocessing
cores = multiprocessing.cpu_count() # per contare il numero di cpu che si hanno
cores

In [ ]:
from torch.utils.data.dataloader import DataLoader
batch_size = 8 # dimensione del batch
train_loader = torch.utils.data.DataLoader(training_set, 
                                           batch_size, 
                                           shuffle = True,
                                           num_workers=cores,
                                           pin_memory=True) # shuffle = True indica che i dati vengono caricati in modo casuale
val_loader = torch.utils.data.DataLoader(validation_set, 
                                         batch_size, 
                                         shuffle = False,
                                         num_workers=cores,  #metto tanti cores quanti ne ho disponibili
                                         pin_memory=True)  # i dati vengono caricati in ordine

### Sposto il DataLoader e i dati sulla GPU

In [ ]:
# Verifico che la GPU sia disponibile
torch.cuda.is_available() # Resistuisce 'True' se la GPU è accessibile

In [ ]:
def get_default_device():
    # Se è disponibile prende la GPU, altrimenti la CPU locale
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
#def to_device(data, device):
#    # Sposta i tensori (dati) sul device stabilito
#        return data.to(device, non_blocking=True)
#    if isinstance(data, torch.Tensor): # sposto i tensori
#    elif isinstance(data, (list,tuple)): # sposto le liste o le tuple
#        return [to_device(x, device) for x in data]
#    elif isinstance(data, dict): # sposto i dizionari
#        return {k: to_device(v, device) for k, v in data.items()}
#    else:
#        return data

def to_device(data, device):
    if isinstance(data, (list,tuple)): 
      return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    # Wrapping del DataLoader per spostare i dati sul device
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        # Restituisce un batch di dati spostandolo sulla GPU
        for b in self.dl: 
            yield to_device(b, self.device) if isinstance(b, torch.Tensor) else b

    def __len__(self):
        # Numero di batch
        return len(self.dl)

In [ ]:
device = get_default_device()
device

In [ ]:
train_loader = DeviceDataLoader(train_loader, device)  # crea un DeviceDataLoader "avvolgendo" (wrapping) il train_loader
val_loader = DeviceDataLoader(val_loader, device) # idem per il val_loader

## Creazionde del modello

In [ ]:
#def set_parameter_requires_grad(model, feature_extracting):
#    if feature_extracting:
#        for param in model.parameters():
#            param.requires_grad = False
#permette di congelare tutti i pesi, non vengono più "aggiustati"

In [ ]:
#resnet18 = torchvision.models.resnet18(pretrained=True)

In [ ]:
#print(resnet18)

In [ ]:
#for param in resnet18.parameters():
#    param.requires_grad = False

In [ ]:
#inizializzo e rimodello le reti dei modelli preaddestrati


In [ ]:
#type(training_set[0].DSC), 
#training_set[0]["rCBV"].data #1: dice il tipo di dato che si ha #2: printa il contenuto del tensore

In [ ]:
#def show_batch_DSC(data_loader):
#  for batch_idx, batch_sample in enumerate(data_loader):
#    images = batch_sample["DSC"][tio.DATA]
#    img_DSC = images[0, 0, :, :, 50] #prendo solo la prima immagine, le x e le y della 50 fetta sulle z del time frame 0.

#    # Visualizzazione delle prime 4 immagini della batch
#    plt.imshow(img_DSC, cmap="gray")
#    plt.show()    # Interrompi l'iterazione dopo la prima batch
#    break

In [ ]:
#show_batch_DSC(train_loader)

In [ ]:
def prepare_batch(batch, device): # estrae l'input e l'output desiderati da una singola batch di dati del loader e li restituisce come tensori PyTorch
    inputs = batch["DSC"][tio.DATA].to(device) # tensore 
    targets = batch["rCBV"][tio.DATA].to(device) # tensore
    return inputs, targets

In [ ]:
from torchmetrics.functional import structural_similarity_index_measure 

In [ ]:
class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        inputs, targets = prepare_batch(batch, device)
        print(inputs.shape)
        print(targets.shape)
        out = self(inputs)  # Generate predictions
        print(out.shape)             
        loss = F.mse_loss(out, targets) # Calculate loss
        print("Fine train")
        return loss

    def validation_step(self, loader):
        inputs, targets = prepare_batch(loader, device)
        out = self(inputs)     # Generate predictions
        loss = F.mse_loss(out, targets)   # Calculate loss

        targets = targets.cpu().detach().numpy().squeeze() #converto in numpy perchè skimage lavora su tensori di questo tipo
        out = out.cpu().detach().numpy().squeeze()
      
        print(targets.shape, out.shape)

        mse_ = np.square(np.subtract(targets, out)).mean() # Calculate mean square error
        nrmse_ = nrmse(targets, out) # Calculate normalized root mean square error
        ssim_ = structural_similarity_index_measure(torch.from_numpy(targets).to(torch.float32), torch.from_numpy(out).to(torch.float32), data_range = out.max() - out.min()) # Calculate structural similarity index
        psnr_ = psnr(targets, out, data_range = out.max() - out.min()) # Calculate peak signal to noise ratio
        return {'val_loss': loss.detach(), 'mse': mse_, 'nrmse': nrmse_, 'ssim': ssim_, 'psnr':psnr_}
   
    def validation_epoch_end(self, outputs):
      batch_losses = [x['val_loss'] for x in outputs]
      epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
      batch_mses = [x['mse'] for x in outputs]
      epoch_mse = np.mean(batch_mses)   # Combine mses
      batch_nrmses = [x['nrmse'] for x in outputs]
      epoch_nrmse = np.mean(batch_nrmses)  # Combine nrmses
      batch_ssims = [x['ssim'] for x in outputs]
      epoch_ssim = np.mean(batch_ssims)   # Combine ssims
      batch_psnrs = [x['psnr'] for x in outputs]
      epoch_psnr = np.mean(batch_psnrs) # Combine psnrs
      return {'val_loss': epoch_loss.item(), 'mse': epoch_mse.item(), 'nrmse': epoch_nrmse.item(), 'ssim': epoch_ssim.item(), 'psnr':epoch_psnr.item()}
    
    def epoch_end(self, epoch, result):
      print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, mse: {:.4f}, nrmse: {:.4f}, ssim: {:.4f}, psnr: {:.4f}".format(epoch+1, result['train_loss'], result['val_loss'], result['mse'], result['nrmse'], result['ssim'], result['psnr']))
  

In [ ]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [ ]:
feature_extract = True

In [ ]:
import torch.nn as nn
import torchvision.models as models

class CustomResNet(ImageClassificationBase):
    def __init__(self, num_classes, time_frame):
        super(CustomResNet, self).__init__()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.encoder = models.video.r3d_18(pretrained=True)
        #set_parameter_requires_grad(self.encoder, feature_extract)
        self.encoder.stem[0] = nn.Conv3d(45, 64, kernel_size=(3, 3, 3), padding=1, bias=False)
        self.decoder = nn.Sequential(
    nn.ConvTranspose3d(in_channels=256, out_channels=128, kernel_size=3, stride=2, padding=1, output_padding=1),
    nn.ConvTranspose3d(in_channels=128, out_channels=1, kernel_size=3, stride=2, padding=1, output_padding=1),
)

    def forward(self, x):
        x = self.encoder.stem(x)
        x = self.encoder.layer1(x)
        x = self.encoder.layer2(x)
        x = self.encoder.layer3(x)
        x = self.encoder.layer4(x)
        x = self.decoder(x)
        return x

In [ ]:
model = CustomResNet(1,45)#.to('cuda')
model.encoder.fc = nn.Sequential() #per eliminare lo strato linere fc
model.encoder.avgpool = nn.Sequential()
model.encoder.layer4 = nn.Sequential()
# sposto il modello sulla GPU
model = to_device(model, device)
#print(model) 
#print('Modello su dispositivo:', next(model.parameters()).device)

## Training del modello

In [ ]:
# definisco una classe per salvare il modello durante il training. 
# se il valore della val_loss corrente è minore del precedente, allora salvo lo stato del modello

class SaveBestModel: 
  def __init__(
      self, best_valid_loss = float('inf')
  ):
      self.best_valid_loss = best_valid_loss

  def __call__(
      self, current_valid_loss,
      epoch, model, optimizer, criterion, path_name #, scheduler
  ): 
      if current_valid_loss < self.best_valid_loss: 
        self.best_valid_loss = current_valid_loss
        print(f"\nBest validation loss: {self.best_valid_loss}")
        print(f"\nSaving best model for epoch: {epoch+1}\n")
        torch.save({
            'epoch': epoch+1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': criterion,
            }, path_name+'.pth')


In [ ]:
@torch.no_grad() #l'utilizzo di @torch.no_grad() all'interno della funzione evaluate() ha senso perché consente di disabilitare il calcolo del gradiente durante la fase di valutazione del modello, migliorando l'efficienza del processo di valutazione.

def evaluate(model, val_loader):
  # with torch.no_grad():
    model.eval()
    print("Evaluate 1 fatto")
    outputs = [model.validation_step(batch) for batch in val_loader]
    print("Evaluate 2 fatto")
    return model.validation_epoch_end(outputs)

#def get_lr(optimizer):
#    for param_group in optimizer.param_groups:
#        return param_group['lr']

#def fit_one_cycle(epochs, max_lr, model, train_loader, val_loader, 
#                  weight_decay=0, grad_clip=None, opt_func=torch.optim.SGD):
#    torch.cuda.empty_cache()
#    history = []
#    
#    # Set up cutom optimizer with weight decay
#    optimizer = opt_func(model.parameters(), max_lr, weight_decay=weight_decay)
#    # Set up one-cycle learning rate scheduler
#    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs, 
#                                                steps_per_epoch=len(train_loader))
#    
#    for epoch in range(epochs):
#        
#        #strt_time = time.time()
#        
#        # Training Phase 
#        model.train()
#        train_losses = []
#        lrs = []
#        for batch in train_loader:
#
#            
#            loss = model.training_step(batch)
#            loss.to(torch.float32)
#            train_losses.append(loss)
#            loss.backward()
#            
#            # Gradient clipping
#            if grad_clip: 
#                nn.utils.clip_grad_value_(model.parameters(), grad_clip)
#            
#            optimizer.step()
#            optimizer.zero_grad()
#            
#            # Record & update learning rate
#            lrs.append(get_lr(optimizer))
#            scheduler.step()
#            
#        #end_time = time.time()
#        #tot_time = end_time-strt_time
#        #print('Epoch training time:', tot_time, 'sec')
#            
#        # Validation phase 
#        result = evaluate(model, val_loader)
#        result['train_loss'] = torch.stack(train_losses).mean().item()
#        result['lrs'] = lrs
#        model.epoch_end(epoch, result)
#        history.append(result)
#        
#        #save_best_model(
#        #result['val_loss'], epoch, model, optimizer, scheduler, criterion, path_name
#    #)
#    return history

def fit(epochs, lr, model, train_loader, val_loader, opt_func, path_name):
    torch.cuda.empty_cache()
    history = []
    optimizer = opt_func(model.parameters(), lr, weight_decay=0.1)
    for epoch in range(epochs):

        # Training Phase 
        print("Inizio del train")
        model.train()
        train_losses = []
        for batch_idx, batch in enumerate(train_loader):
            loss = model.training_step(batch).detach().requires_grad_(True) #uso detach per provare a creare un nuovo tensore che possa fare il gradient descend
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        # Validation phase
        print("Inizio fase di validazione")
        result = evaluate(model, val_loader)
        print("Evaluate fatto")
        result['train_loss'] = torch.stack(train_losses).mean().item()
        print("2 fatto")
        model.epoch_end(epoch, result)
        print("3 fatto")
        history.append(result)

        save_best_model(
            result['val_loss'], epoch, model, optimizer, criterion, path_name
        )

    return history

In [ ]:
num_epochs = 10
opt_func = torch.optim.Adam
lr=1e-2

In [ ]:
print(model)

In [ ]:
summary(model.cuda(), input_size=(2,45,60,60,40))


In [ ]:
evaluate(model, val_loader)

In [ ]:
#history1 = []
#history1 += fit_one_cycle(40, 0.01, model, train_loader, val_loader, weight_decay=0.01, grad_clip=20, opt_func=torch.optim.Adam)

In [ ]:
# loss function
criterion = nn.MSELoss()

# optimizer 
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001) #weight decay = 0.1

# scheduler (none)

# initialize SaveBestModel class
save_best_model = SaveBestModel()


In [ ]:
history = fit(num_epochs, lr, model, train_loader, val_loader, opt_func, path_name='ResNet_best_model_train')

In [ ]:
def plot_val_loss(history):
    val_loss = [x['val_loss'] for x in history]
    plt.plot(val_loss, '-x')
    plt.xlabel('epoch')
    plt.ylabel('val_loss')
    plt.title('val_loss vs. No. of epochs')
    plt.show();

In [ ]:
def plot_train_loss(history):
    train_loss = [x['train_loss'] for x in history]
    plt.plot(train_loss, '-x')
    plt.xlabel('epoch')
    plt.ylabel('train_loss')
    plt.title('train_loss vs. No. of epochs')
    plt.show();

In [ ]:
def plot_MSE(history):
    mse = [x['mse'] for x in history]
    plt.plot(mse, '-x')
    plt.xlabel('epoch')
    plt.ylabel('MSE')
    plt.title('MSE vs. No. of epochs')
    plt.show();

In [ ]:
def plot_NRMSE(history):
    nrmse = [x['nrmse'] for x in history]
    plt.plot(nrmse, '-x')
    plt.xlabel('epoch')
    plt.ylabel('NRMSE')
    plt.title('NRMSE vs. No. of epochs')
    plt.show();

In [ ]:
def plot_PSNR(history):
    psnr = [x['psnr'] for x in history]
    plt.plot(psnr, '-x')
    plt.xlabel('epoch')
    plt.ylabel('PSNR')
    plt.title('PSNR vs. No. of epochs')
    plt.show();

In [ ]:
plot_val_loss(history), plot_train_loss(history), plot_MSE(history), plot_NRMSE(history), plot_PSNR(history)

## Testing del modello

In [ ]:
def predict_image(img, model):
    # Convert to a batch of 1
    xb = img  #.unsqueeze(0)
    # Get predictions from model
    yb = model(xb)
    #

In [ ]:
#visualizzare un'immagine specifica
training_set[0].rCBV
#nii_array = nii_image.numpy() #bisogna trasformare l'immagin ein vettore con il comando numpy()
#plt.imshow(nii_array[1, :, :, 50], cmap='gray') #time_frame,x,y,z
#plt.show()
#training_set[0].DSC, training_set[0].rCBV

In [ ]:
dataset[0]["DSC"].plot()

In [ ]:
xb = dataset[0]["rCBV"][tio.DATA]
print(xb.shape)
plt.imshow(xb[0, ...,90])

In [ ]:
import random
subject = random.choice(validation_set)

input_tensor = subject['DSC'][tio.DATA].to(device)
input_tensor.shape

model.eval()

with torch.no_grad():
    pred = model(input_tensor.unsqueeze(0).to(torch.float32)) 
    plt.imshow(pred[0,0, ..., 30].cpu())

In [ ]:
pred.shape
input_tensor.shape

In [ ]:
print(dataset[0]["DSC"][tio.DATA].shape)
yb=dataset[0]["DSC"][tio.DATA].unsqueeze(dim=0)
print(yb.shape)
yb=model(dataset[0]["DSC"][tio.DATA].unsqueeze(dim=0).cuda().to(torch.float32))
plt.imshow(yb.detach().numpy()[0,0, :, :, 50], cmap='gray')

In [ ]:
torch.cuda.empty_cache()
device = torch.device("cpu")
input_tensor = dataset[0]["DSC"][tio.DATA].cpu().to(torch.float32).unsqueeze(dim=0)#.to(model.device).to(torch.float32)
yb = model(input_tensor)
plt.imshow(yb.detach().cpu().numpy()[0, 0, :, :, 80], cmap='gray')

Funziona, èquella presentata al secondo round. Le metriche ottenute non sono buone. Va migliorato il modello.